# ambiguousReads.ipynb
## Marcus Viscardi,    July 03, 2023
***
The mains goals of this file are to test out how I want to handle ambiguous reads. There are two main ways I can go about this:

1. We could just remove other isoforms for the dataset, thus "collapsing" the transcriptome into NMD-sensitive and NMD-insensitive isoforms
2. We could try and identify the unique regions of the NMD-sensitive isoforms, so if there is any coverage on those nucleotides, it would be considered NMD-sensitive. This should be more accurate, but require some more serious coding.

Overall this is going to require a bit of work. I think I am first going to try to implement the first (#1) solution. As that should be a little quicker. Then we can assess if that is good enough, or if we need to go further (with solution #2).

***
A lot of this is going to be leaning on code from 'coverageAndTails_wFlairAssignments.ipynb' and modifying it with the above goals in mind!
***
One of the main takeaways from my old dataloading method was I had to squeeze the FLAIR results a bit to make it work for me. My option here could honestly be to manually assign reads to either the isoform of interest or not...

In [ ]:
# Old imports from coverageAndTails_wFlairAssignments.ipynb:
import pysam
import pandas as pd
from pathlib import Path
import sys
sys.path.insert(0, '/data16/marcus/scripts/nanoporePipelineScripts')
import nanoporePipelineCommon as npCommon
from nanoporeReadPlotting.finalizingReadAndCoveragePlotting_matplotlib import plot_reads, coverage_plotting_5tera
CONVERSION_DICT = {"xrn-1-5tera": "oldN2",
                   "xrn-1-5tera-smg-6": "oldS6",
                   "5tera_xrn-1-KD_wt": "newN2",
                   "5tera_xrn-1-KD_wt_rerun": "newerN2",
                   "5tera_xrn-1-KD_smg-6_rerun": "newerS6",
                   "5tera_xrn-1-KD_smg-5_rerun": "newerS5",
                   "5tera_xrn-1-KD_smg-5": "newS5",
                   "5tera_xrn-1-KD_smg-6": "newS6",
                   "5tera_xrn-1-KD_smg-7": "newS7",
                   "sPM57": "sPM57",
                   "sPM58": "sPM58",
                   }
REV_CONVERSION_DICT = {val: key for key, val in CONVERSION_DICT.items()}
LIB_NAMES = list(REV_CONVERSION_DICT.keys())

output_dir = "/data16/marcus/working/230703_NMD_readCategorization" # or /tmp

print(f"Finished imports at: {npCommon.get_dt(for_print=True)}")

# Ideas:

Let's try to logic through what would be required to do the isoform assignment solly based on the knowledge of which piece of the genome is unique to the NMD-sensitive species. Some initial musings:

Would could use a region of coverage to call a read as sensitive or not. The idea would be to calculate coverage for all reads that were assigned to the gene based on FeatureCounts. Then look at the coverage of each read and see if it overlaps with the NMD-sensitive region we identified. If it does, then it is NMD-sensitive, if not, then it is NMD-insensitive.

I *could* do this manually, but I think pysam actual has a tool to do this. I think it is called `pysam.fetch()`. I think I can use this to get reads that hit the NMD-sensitive region, then I can use that plus the end of the read to decide if it is NMD-sensitive or not.

Limitations (and initial thoughts on how to address them):
1. For reads that are genuinely ambiguous because they are not long enough to have coverage, we can look at the mapped "end" of the read, if that spanned past the NMD-sensitive region, then we can call it. If it didn't, then it is a **TRUE ambiguous read**!
2. If a gene has an NMD isoform that is a skipped region rather than an included region, IDK if we can do the same process. This would be pretty annoying. We could potentially just have it work the opposite way, the function would take a inclusive_or_exclusive tag. So that if something hits the "identifying" region of the transcript it would be either NMD-sensitive or not, depending on our input. This would be a bit annoying, but I think it would work.
3. IDK how well I could double filter with pysam. B/c I would want to select reads identified by FeatureCounts, THEN identified reads with or without coverage at the "identifying" region.


First let's try to see how we can use pysam to get reads that hit a region of the genome. I think we can use `pysam.fetch()` to do this. Let's try it out:

In [ ]:
gtf_df = pd.read_parquet('/data16/marcus/genomes/plus_cerENO2_elegansRelease100/230327_allChrs_plus-cerENO2.gtf.parquet')
from ambiguousReads import NMD_ISO_REGIONS
targets_df = pd.DataFrame.from_dict(NMD_ISO_REGIONS,
                                    orient='index').reset_index().rename(columns={'index': 'gene_name',
                                                                                  'start': 'nmd_region_start',
                                                                                  'end': 'nmd_region_end',
                                                                                  'region_is_target': 'nmd_region_is_target'})

targets_df = targets_df.merge(gtf_df[gtf_df['gene_name'].isin(targets_df.gene_name) & (gtf_df['feature'] == 'gene')][['gene_name', 'gene_id', 'chr', 'start', 'end', 'strand']])
targets_df = targets_df[['gene_name', 'gene_id', 'chr', 'start', 'end', 'strand', 'nmd_region_start', 'nmd_region_end', 'nmd_region_is_target']]
targets_df.to_csv(f"{output_dir}/identifiable_targets_df.csv", index=False)
print(f"Saving to: {output_dir}/identifiable_targets_df.csv")
targets_df

# Categorize based on coverage of an "NMD-region" (i.e. a region that defines the NMD-sensitive isoform)

This works ***amazingly well***

Look for the output bam (and index) in the /tmp directory, load that into IGV, and color by tag: nC

You can see that the reads are properly filtered for all the genes in the above list!

In [ ]:
target_gene = 'ubl-1'

chr, start, end, strand, nmd_start, nmd_end = targets_df.loc[targets_df['gene_name'] == target_gene, ['chr', 'start', 'end', 'strand', 'nmd_region_start', 'nmd_region_end']].values[0]
gene_is_reverse = (strand == '-')
last_edge = nmd_start if gene_is_reverse else nmd_end
comparison_for_edge = f"<= {nmd_start}" if gene_is_reverse else f">= {nmd_end}"

bam_file = Path(npCommon.pick_lib_return_path(REV_CONVERSION_DICT['newerS5'], output_dir_folder='cat_files', file_midfix='cat.sorted.mappedAndPrimary', file_suffix='.bam'))

bam = pysam.AlignmentFile(bam_file, 'rb')
# Let's make temporary bam files for each of the below groups:
# 1. Reads that hit the NMD-sensitive region
# 2. Reads that did not hit the NMD-sensitive region
# 3. Reads that were ambiguous
general_name = f'{output_dir}/{bam_file.parent.parent.parent.stem}.{target_gene}'
print(general_name)
nmd_bam_path = Path(f'{general_name}.nmd.bam')
nmd_bam = pysam.AlignmentFile(nmd_bam_path, 'wb', template=bam)
non_nmd_bam_path = Path(f'{general_name}.non_nmd.bam')
non_nmd_bam = pysam.AlignmentFile(non_nmd_bam_path, 'wb', template=bam)
ambiguous_bam_path = Path(f'{general_name}.ambiguous.bam')
ambiguous_bam = pysam.AlignmentFile(ambiguous_bam_path, 'wb', template=bam)


nmd_count = 0
non_nmd_count = 0
ambiguous_count = 0
for i, read in enumerate(bam.fetch(chr, start, end)):
    if read.is_reverse != gene_is_reverse:
        continue
    # if eval(f"read.reference_start {comparison_for_edge}"):
    #     ambiguous_count += 1
    #     read.set_tag('nC', 'ambiguous', value_type='Z')
    #     ambiguous_bam.write(read)
    if gene_is_reverse and eval(f"read.reference_end <= {nmd_start}"):
        ambiguous_count += 1
        read.set_tag('nC', 'ambiguous', value_type='Z')
        ambiguous_bam.write(read)
    elif not gene_is_reverse and eval(f"read.reference_start >= {nmd_end}"):
        ambiguous_count += 1
        read.set_tag('nC', 'ambiguous', value_type='Z')
        ambiguous_bam.write(read)
    elif read.get_overlap(nmd_start, nmd_end) >= 5:
        nmd_count += 1
        read.set_tag('nC', 'nmd_target', value_type='Z')
        nmd_bam.write(read)
    else:
        non_nmd_count += 1
        read.set_tag('nC', 'non_nmd_target', value_type='Z')
        non_nmd_bam.write(read)

print(f"\nGene: {target_gene}\nnmd_count: {nmd_count:>13,}\nnon_nmd_count: {non_nmd_count:>9,}\nambig_count: {ambiguous_count:>11,}\ntotal_count: {nmd_count + ambiguous_count +non_nmd_count:>11,}")

for bam in [nmd_bam, non_nmd_bam, ambiguous_bam]:
    bam.close()
    pysam.index(bam.filename)  # These samtools methods aren't in the pysam __init__ file, but they work!

pysam.merge('-f', f'{general_name}.merge.bam', '--write-index', str(nmd_bam_path), str(non_nmd_bam_path), str(ambiguous_bam_path))

In [ ]:
bam_file = Path(npCommon.pick_lib_return_path(REV_CONVERSION_DICT['newerN2'], output_dir_folder='cat_files', file_midfix='cat.sorted.mappedAndPrimary', file_suffix='.bam'))

bam = pysam.AlignmentFile(bam_file, 'rb')
# Let's make temporary bam files for each of the below groups:
# 1. Reads that hit the NMD-sensitive region
# 2. Reads that did not hit the NMD-sensitive region
# 3. Reads that were ambiguous
general_name = f'{output_dir}/{bam_file.parent.parent.parent.stem}.all'
nmd_bam_path = Path(f'{general_name}.nmd.bam')
nmd_bam = pysam.AlignmentFile(nmd_bam_path, 'wb', template=bam)
non_nmd_bam_path = Path(f'{general_name}.non_nmd.bam')
non_nmd_bam = pysam.AlignmentFile(non_nmd_bam_path, 'wb', template=bam)
ambiguous_bam_path = Path(f'{general_name}.ambiguous.bam')
ambiguous_bam = pysam.AlignmentFile(ambiguous_bam_path, 'wb', template=bam)



for gene_name in targets_df['gene_name'].unique():
    chr, start, end, strand, nmd_start, nmd_end = targets_df.loc[targets_df['gene_name'] == gene_name,
    ['chr', 'start', 'end', 'strand', 'nmd_region_start', 'nmd_region_end']].values[0]
    
    gene_is_reverse = (strand == '-')
    gene_is_forward = not gene_is_reverse
    last_edge = nmd_start if gene_is_reverse else nmd_end
    comparison_for_edge = f"<= {nmd_start}" if gene_is_reverse else f">= {nmd_end}"
    
    nmd_count = 0
    non_nmd_count = 0
    ambiguous_count = 0
    
    for i, read in enumerate(bam.fetch(chr, start, end)):
        if read.is_reverse != gene_is_reverse:
            continue
        # if eval(f"read.reference_start {comparison_for_edge}"):
        #     ambiguous_count += 1
        #     read.set_tag('nC', 'ambiguous', value_type='Z')
        #     ambiguous_bam.write(read)
        if gene_is_reverse and eval(f"read.reference_end <= {nmd_start}"):
            ambiguous_count += 1
            read.set_tag('nC', 'ambiguous', value_type='Z')
            ambiguous_bam.write(read)
        elif not gene_is_reverse and eval(f"read.reference_start >= {nmd_end}"):
            ambiguous_count += 1
            read.set_tag('nC', 'ambiguous', value_type='Z')
            ambiguous_bam.write(read)
        elif read.get_overlap(nmd_start, nmd_end) >= 5:
            nmd_count += 1
            read.set_tag('nC', 'nmd_target', value_type='Z')
            nmd_bam.write(read)
        else:
            non_nmd_count += 1
            read.set_tag('nC', 'non_nmd_target', value_type='Z')
            non_nmd_bam.write(read)
    
    print(f"Gene: {gene_name}\n\tnmd_count: {nmd_count:>13,}\n\tnon_nmd_count: {non_nmd_count:>9,}\n\tambig_count: {ambiguous_count:>11,}\n\ttotal_count: {nmd_count + ambiguous_count +non_nmd_count:>11,}")

for bam in [nmd_bam, non_nmd_bam, ambiguous_bam]:
    bam.close()
    pysam.sort(bam.filename, '-o', bam.filename)
    pysam.index(bam.filename)  # These samtools methods aren't in the pysam __init__ file, but they work!

pysam.merge('-f', f'{general_name}.merge.bam', str(nmd_bam_path), str(non_nmd_bam_path), str(ambiguous_bam_path))
pysam.sort(f'{general_name}.merge.bam', '-o', f'{general_name}.merge.bam')
pysam.index(f'{general_name}.merge.bam')

In [ ]:
read.__dir__()